## Evaluación con distancia de Levenshtein.





#### Construcción del target
 Antes de abordar el problema de la evaluación de los outputs, vamos a extraer entidades de los achivos transcriptos a mano. De esa manera, construimos nuestro target para poder comparar. Utilizamos la misma lógica de extracción de entidades que en los otros casos y usamos el modelo Llama 3. 

In [4]:
from pydantic import BaseModel
from typing import Optional
from pydantic import ValidationError
import re, unicodedata
from tqdm import tqdm
from requests.exceptions import ReadTimeout
from pathlib import Path
import json
import csv
import requests
from requests.exceptions import ReadTimeout

In [5]:
TXT_FOLDER = r"D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo\Transcripciones_manual"
OUTPUT_CSV = "entidades_extraidas_mauscritas.csv"
PROMPT_FILE = "prompt_2.txt"
MODEL       = "llama3:latest"
OLLAMA_URL  = "http://localhost:11434/api/generate"
TEMPERATURE = 0.0
TIMEOUT     = 800
MAX_CHARS   = 9000

In [6]:
class Entity(BaseModel):
    Remitente: Optional[str] = None
    DNI: Optional[str] = None
    CUIT_CUIL: Optional[str] = None
    Cuerpo: str


In [7]:
CLEAN_RE = re.compile(r"[^\w\s.,;:()@€$%/-]")

def clean_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = CLEAN_RE.sub("", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

In [8]:
JSON_SCHEMA = {
    "type": "object",
    "properties": {
        "Remitente": {"type": ["string", "null"]},
        "DNI": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "CUIT_CUIL": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "Cuerpo": {"type": "string"},
    },
    "required": ["Cuerpo"],
    "additionalProperties": False,
}

with open(PROMPT_FILE, encoding="utf-8") as f:
    PROMPT = f.read()

def call_ollama(text: str) -> dict:
    payload = {
        "model": MODEL,
        "prompt": f"<|system|>\n{PROMPT}<|end|>\n<|user|>\n{text[:MAX_CHARS]}<|end|>\n<|assistant|>",
        "format": "json",
        "stream": False,
        "options": {"temperature": TEMPERATURE, "json_schema": JSON_SCHEMA},
    }
    try:
        r = requests.post(OLLAMA_URL, json=payload, timeout=TIMEOUT)
    except ReadTimeout:
        raise TimeoutError("Timeout de Ollama")
    r.raise_for_status()
    return json.loads(r.json()["response"])

def main():
    dir_path = Path(TXT_FOLDER)
    if not dir_path.is_dir():
        raise SystemExit(f"Ruta no encontrada: {dir_path}")

    rows = []
    for file in tqdm(sorted(dir_path.glob("*.txt")), desc="TXT"):
        raw = file.read_text(encoding="utf-8", errors="ignore")
        cleaned = clean_text(raw)
        try:
            data = call_ollama(cleaned)
            ent = Entity.model_validate(data)
            rows.append({
                "ARCHIVO": file.name,
                "Remitente": (ent.Remitente or "NaN").strip() or "NaN",
                "DNI": re.sub(r"\D", "", ent.DNI or "") or "NaN",
                "CUIT_CUIL": re.sub(r"\D", "", ent.CUIT_CUIL or "") or "NaN",
                "Cuerpo": ent.Cuerpo.strip() or "NaN",
            })
        except Exception as e:
            print(f"❌ {file.name}: {e}")
    if rows:
        with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
            csv.DictWriter(f, fieldnames=rows[0].keys()).writeheader(); csv.DictWriter(f, fieldnames=rows[0].keys()).writerows(rows)
        print(f"✅ CSV generado en {OUTPUT_CSV} ({len(rows)} filas)")
    else:
        print("No se extrajeron entidades válidas.")

In [ ]:
if __name__ == "__main__":
    main()

TXT:  57%|█████▋    | 4/7 [20:06<14:28, 289.53s/it]

### Evalaución

Una vez realizada la estracción de entidades con cada método, vamos a analizar cual funcionó mejor. Para ello, vamos a utilizar la **distancia de Levenshtein**. Esta medida indica cuántas operaciones mínimas se necesitan para transformar una cadena en otra. Las operaciones posibles son:
 - Inserción de un carácter.
 - Eliminación de un carácter.
 - Sustitución de un carácter por otro.

Por lo tanto, obtener un valor cero significa coincidencia exacta. Cuanto mayor sea el número, más diferentes son las cadenas.

Para cada archivo CSV:
- Normalizamos el nombre y el contenido para evitar diferencias por formato, mayúsculas o caracteres especiales.
- Extraemos variables clave: cuerpo de la carta, DNI y CUIT.
- Validamos el CUIT mediante el algoritmo de dígito verificador (módulo 11).
- Buscamos el archivo correspondiente en cada CSV usando coincidencia por nombre normalizado.
- Comparamos el contenido del TXT con el CSV calculando la distancia de Levenshtein.

La salida final muestra, para cada archivo de la base, las distancias entre lo que dice la base y lo que dice cada target.
Los valores van de 0 a 1, donde 0 = idéntico y 1 = completamente distinto

La ventaja de usar Levenshtein es que detecta pequeñas diferencias (errores de OCR, faltas de ortografía, caracteres faltantes) sin marcar el texto como “completamente distinto”, lo que es clave cuando se trabaja con documentos escaneados y reconocimiento de texto.

Importante: Para correr esta parte del script tenés que instalar jellyfish en el entorno con *pip install pandas jellyfish*


1. Instalamos librerías necesarias

In [1]:
from __future__ import annotations
import sys
import re
import unicodedata
from pathlib import Path
from typing import Optional, Dict

import pandas as pd

try:
    import jellyfish
except ModuleNotFoundError:
    raise SystemExit(
        f"[ERROR] jellyfish no está instalado en este intérprete:\n  {sys.executable}\n"
        f"Instalá con:\n  \"{sys.executable}\" -m pip install jellyfish"
    )

2. Definimos el path

In [2]:
BASE_DIR = r"D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo"
CSV_TARGET_MANUS = Path(BASE_DIR) / "entidades_extraidas_mauscritas.csv"  # <- target
CSV_BASES = {  # <- bases (OCR)
    "DOCTR": Path(BASE_DIR) / "entidades_extraidas_DOCTR.csv",
    "GPT": Path(BASE_DIR) / "entidades_extraidas_GPT.csv",
    "Tesseract": Path(BASE_DIR) / "entidades_extraidas_Tesseract.csv",
}

3. Definimos funciones de normalización para cada variable, para hacerlas más comparables entre sí.

    **1) `quitar_acentos(text)`**

    * Convierte el texto a forma Unicode **NFD** y quita los caracteres de marca diacrítica (`Mn`).
    * Resultado: el string **sin tildes/acentos**.

    **2) Variable cuerpo: `norm_cuerpo(x)`**

    * Normaliza texto de **cuerpo** para comparaciones:
    a) quita acentos → `lower()`
    b) elimina todo lo que **no sea** letras minúsculas `a–z`, dígitos `0–9` o espacio (`\s`) mediante regex `[^a-z0-9\s]`
    c) colapsa espacios múltiples a **uno**.


    **3) Variable nombre: `norm_nombre(x)`**

    * Normaliza **nombres** (Remitente, por ejemplo):
    a) quita acentos → `lower()`
    b) **deja solo letras** y espacios (`[^a-z\s]`)
    c) colapsa espacios.


    **4)VARIABLES DNI y CUIT_CUIL:`norm_num(x)`**
    * Normaliza **números** (DNI, CUIT):
    a) convierte a `str`, hace `strip()`
    b) si termina en `.0` (típico de CSV/Excel), lo recorta
    c) remueve **todo lo que no sea dígito** con `[^0-9]`
    d) devuelve `None` si quedó vacío.

    **5) `norm_fname(s)`**

    * Genera la **clave de emparejamiento** por nombre de archivo:
    a) toma solo el **nombre** (`Path(...).name`) y el **stem** (sin extensión)
    b) quita acentos, pasa a minúsculas
    c) elimina **todo** lo que no sea `a–z` o `0–9`.

    **6) `lev(a, b)`**

    * Calcula la **distancia de Levenshtein** entre `a` y `b` usando **jellyfish**.
    * Si alguno es `None`, devuelve `None`.
    * 0 = idéntico; valores mayores = más diferencias.
    * Se usa sobre las **formas normalizadas** (de arriba) para comparaciones justas.


In [3]:
def quitar_acentos(text: str) -> str:
    if text is None:
        return ""
    return "".join(c for c in unicodedata.normalize("NFD", str(text)) if unicodedata.category(c) != "Mn")


def norm_cuerpo(x: str) -> str:
    if not x:
        return ""
    t = quitar_acentos(x).lower()
    t = re.sub(r"[^a-z0-9\s]", "", t)
    return " ".join(t.split())


def norm_nombre(x: str) -> str:
    if not x:
        return ""
    t = quitar_acentos(x).lower()
    t = re.sub(r"[^a-z\s]", "", t)
    return " ".join(t.split())


def norm_num(x) -> Optional[str]:
    if x is None:
        return None
    s = str(x).strip()
    if s.endswith('.0'):
        s = s[:-2]
    s = re.sub(r"[^0-9]", "", s)
    return s if s else None


def norm_fname(s: str) -> str:
    """Clave de match:** stem** sin acentos, minúsculas, sólo [a-z0-9]."""
    base = Path(str(s)).name
    stem = Path(base).stem
    t = quitar_acentos(stem).lower()
    return re.sub(r"[^a-z0-9]", "", t)


def lev(a: Optional[str], b: Optional[str]) -> Optional[int]:
    if a is None or b is None:
        return None
    return jellyfish.levenshtein_distance(a, b)

Funciones de lectura: Estas dos funciones homogeneizan los CSV de entrada y construyen una clave de emparejamiento por nombre de archivo. Esto permite que todos los datasets compartan el mismo esquema de columnas y una clave (archivo_norm) comparable entre sí.

In [4]:
def estandarizar_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={c: (c.strip() if isinstance(c, str) else c) for c in df.columns})
    lower = {str(c).lower(): c for c in df.columns}

    if 'archivo' in lower:
        df = df.rename(columns={lower['archivo']: 'archivo'})
    elif 'ARCHIVO' in df.columns:
        df = df.rename(columns={'ARCHIVO': 'archivo'})

    if 'remitente' in lower and 'Remitente' not in df.columns:
        df = df.rename(columns={lower['remitente']: 'Remitente'})
    if 'cuerpo' in lower and 'Cuerpo' not in df.columns:
        df = df.rename(columns={lower['cuerpo']: 'Cuerpo'})
    if 'dni' in lower and 'DNI' not in df.columns:
        df = df.rename(columns={lower['dni']: 'DNI'})
    for k in ['cuit_cuil','cuitcuil','cuit','cuil']:
        if k in lower and 'CUIT_CUIL' not in df.columns:
            df = df.rename(columns={lower[k]: 'CUIT_CUIL'})
            break
    if 'CUIT_CUIL' not in df.columns and 'Cuit_Cuil' in df.columns:
        df = df.rename(columns={'Cuit_Cuil': 'CUIT_CUIL'})

    return df


def cargar_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, encoding='utf-8-sig', dtype=str)
    df = estandarizar_cols(df)
    req = {'archivo','Remitente','DNI','CUIT_CUIL','Cuerpo'}
    faltan = req - set(df.columns)
    if faltan:
        raise ValueError(f"Faltan columnas en {path.name}: {faltan}. Encontradas: {list(df.columns)}")
    # limpiar filas sin archivo
    df = df[df['archivo'].notna() & (df['archivo'].astype(str).str.strip()!='')].copy()
    df['archivo_norm'] = df['archivo'].map(norm_fname)
    return df



Esta función compara fila a fila, un dataset base (p. ej. DOCTR/GPT/Tesseract) contra el target (manuscritas) y calculaa distancias de Levenshtein por campo: Remitente, Cuerpo, DNI, CUIT_CUIL.

In [5]:
def comparar_base_con_target(df_base: pd.DataFrame, df_target: pd.DataFrame, nombre_base: str) -> pd.DataFrame:
    """Left join: mantenemos **todas las filas de la base** y buscamos su par en el target."""
    b = df_base[['archivo','Remitente','DNI','CUIT_CUIL','Cuerpo','archivo_norm']].copy()
    t = df_target[['Remitente','DNI','CUIT_CUIL','Cuerpo','archivo_norm']].copy()

    m = b.merge(t, on='archivo_norm', how='left', suffixes=('_base','_tgt'))

    dist_rem, dist_cue, dist_cue_norm, dist_dni, dist_cuit = [], [], [], [], []
    for _, r in m.iterrows():
        # Remitente
        rb = norm_nombre(r.get('Remitente_base'))
        rt = norm_nombre(r.get('Remitente_tgt')) if pd.notna(r.get('Remitente_tgt')) else None
        drem = lev(rb, rt); dist_rem.append(pd.NA if drem is None else drem)
        # Cuerpo
        cb = norm_cuerpo(r.get('Cuerpo_base'))
        ct = norm_cuerpo(r.get('Cuerpo_tgt')) if pd.notna(r.get('Cuerpo_tgt')) else None
        dcue = lev(cb, ct); dist_cue.append(pd.NA if dcue is None else dcue)
        if ct is None:
            dist_cue_norm.append(pd.NA)
        else:
            mlen = max(len(cb), len(ct)); dist_cue_norm.append(pd.NA if mlen==0 else dcue/mlen)
        # DNI
        db = norm_num(r.get('DNI_base'))
        dt = norm_num(r.get('DNI_tgt')) if pd.notna(r.get('DNI_tgt')) else None
        ddni = lev(db, dt); dist_dni.append(pd.NA if ddni is None else ddni)
        # CUIT
        qb = norm_num(r.get('CUIT_CUIL_base'))
        qt = norm_num(r.get('CUIT_CUIL_tgt')) if pd.notna(r.get('CUIT_CUIL_tgt')) else None
        dcui = lev(qb, qt); dist_cuit.append(pd.NA if dcui is None else dcui)

    out = pd.DataFrame({
        'ARCHIVO': m['archivo'],
        'base': nombre_base,  # <- DOCTR/GPT/Tesseract
        'dist_remitente': pd.Series(dist_rem, dtype='Int64'),
        'dist_cuerpo': pd.Series(dist_cue, dtype='Int64'),
        'dist_cuerpo_norm': pd.Series(dist_cue_norm, dtype='Float64'),
        'dist_dni': pd.Series(dist_dni, dtype='Int64'),
        'dist_cuit': pd.Series(dist_cuit, dtype='Int64'),
    })
    return out

Definimos el orquetador de las funciones y ejecutamos.

In [6]:
def comparar_bases_vs_target(csv_target: Path = CSV_TARGET_MANUS, csv_bases: Dict[str, Path] = CSV_BASES) -> pd.DataFrame:
    df_target = cargar_csv(csv_target)
    resultados = []
    for nombre_base, ruta in csv_bases.items():
        df_base = cargar_csv(ruta)
        df_cmp = comparar_base_con_target(df_base, df_target, nombre_base)
        if not df_cmp.empty:
            resultados.append(df_cmp)
    if not resultados:
        return pd.DataFrame(columns=['ARCHIVO','base','dist_remitente','dist_cuerpo','dist_cuerpo_norm','dist_dni','dist_cuit'])
    return pd.concat(resultados, ignore_index=True)


if __name__ == "__main__":
    df_out = comparar_bases_vs_target()
    # Orden por archivo y base
    df_out = df_out.sort_values(['ARCHIVO','base'], kind='mergesort')
    print(f"[OK] Comparadas {df_out['base'].nunique()} bases contra target manuscritas, sobre {df_out['ARCHIVO'].nunique()} archivos.")
    print(df_out.head(20).to_string(index=False))

[OK] Comparadas 3 bases contra target manuscritas, sobre 13 archivos.
                ARCHIVO      base  dist_remitente  dist_cuerpo  dist_cuerpo_norm  dist_dni  dist_cuit
       Belen_payway.txt     DOCTR            <NA>         <NA>              <NA>      <NA>       <NA>
       Belen_payway.txt       GPT            <NA>         <NA>              <NA>      <NA>       <NA>
       Belen_payway.txt Tesseract            <NA>         <NA>              <NA>      <NA>       <NA>
      Bruno23689270.txt     DOCTR               3          875          0.300791      <NA>       <NA>
      Bruno23689270.txt       GPT              15          624          0.274165         9       <NA>
      Bruno23689270.txt Tesseract               3          306          0.127076      <NA>       <NA>
  Bruno400212294002.txt     DOCTR               7           33          0.017638         4       <NA>
  Bruno400212294002.txt       GPT               0           39          0.021047         4       <NA>
  Bruno40021